## Freni-Sterrantino et al 2017 - BYM2 connected, disconnected for Scotland Lip Cancer Dataset

Mitzi Morris

In [A note on intrinsic Conditional Autoregressive models for disconnected graphs](https://arxiv.org/abs/1705.04854), Freni-Sterrantino et.al. show how to implement the BYM2 model for use with areal data where the graph structure of the map is not fully connected.  In this notebook, we present that Stan implementation of this proposal.

### Disease mapping: computing relative risk over a map of geographical regions

Disease mapping concerns the study of disease risk over a map of geographical regions.
For an areal map of $I$ regions, the outcome $y_i$ is the number of cases of a given disease in region $i$. For a rare disease, a Poisson model is assumed, $y_i | {\theta}_i \sim Po({\theta}_i)$ with mean ${\theta}_i = E_i * r_i$, where $E_i$ is the expected cases count for the disease and $r_i$ is the relative risk.  Relative risk values above 1 indicate higher risk associated living a region. The relative risk can be modelled in terms of the effect of a covariates X as $log(r_i) = \alpha + \beta * x_i + {re}_i$; $\alpha$ is the baseline log risk, $\beta$ is the effect of the covariates, and ${re}_i$ is a random effect capturing extra Poisson variability possibly due to unobserved risk factors.

In these models, areal maps are represented as a graph where the nodes in the graph are areal regions and the undirected edges in the graph represent the symmetric neighbor relationship.

### The BYM2 model

The BYM2 model is a disease mapping model presented in [Riebler et al. 2016](https://arxiv.org/abs/1601.0118).
For the above disease mapping regression model, the random effects component is parameterized in terms of:

- $\phi$, an ICAR component which accounts for the spatial structure of the data.
- $\theta$, an ordinary random effects component which accounts for non-spatial heterogeneity.
- $\rho$,  a mixing parameter which accounts for the amount of spatial/non-spatial variation.
- $\sigma$, a precision (scale) parameter placed on the combined ICAR and ordinary random effects components.

In order for $\sigma$ to legitimately be the standard deviation of the combined components,
it is critical that for each $i$, $\operatorname{Var}(\phi_i) \approx \operatorname{Var}(\theta_i) \approx 1$. therfore, the BYM2 model introduces a scaling factor $\tau$ to the model.
Riebler recommends scaling the ICAR component $\phi$ so the geometric mean of the average marginal variance of its elements is 1. 
By dividing the spatial component $\phi$ by $\sqrt{\tau}$, the variances of these components are on the same scale.
For irregular areal maps, where individual regions have varying number of neighbors, the scaling factor $\tau$ necessarily comes into the model as data.

The combined random effects component for the BYM2 model is: 
$$\sigma (\sqrt{1-\rho}\,\theta^* + (\sqrt{\rho/\tau}\,\phi^* )$$

The recommended priors are:

- A standard prior on the standard deviation $\sigma$; we use a half-normal, also possible are half-t or an exponential.
- A beta(1/2,1/2) prior on $\rho$.

The Stan case study [Spatial Models in Stan: Intrinsic Auto-Regressive Models for Areal Data](https://mc-stan.org/users/documentation/case-studies/icar_stan.html) provides the background and derivations for the ICAR and the BYM2 model.

### Stan implementation of the BYM2 model for a fully connected spatial structure

For the Stan implementation of the ICAR component, we compute the per-node spatial variance by representing the spatial structrue of the map as an _edgelist_; a 2D array of size 2 × J where J is the number of edges in the graph. Each column entry in this array represents one undirected edge in the graph, where for each edge j, entries [j,1] and [j,2] index the nodes connected by that edge. Treating these are parallel arrays and using Stan's vectorized operations provides a transparent implementation of the pairwise difference formula used to compute the ICAR component.


When the areal map is a single, fully connected component, i.e., a graph where any node in the graph can be reached from any other node, the BYM2 model is implemented as follows.
The spatial structure and scaling factor are input as data, therefore declared in the model's `data` block:

The spatial and heterogeneous effects, combined variance, and proportion of spatial variance are model parameters, defined in the `parameters` block:

The combined random effects component is computed in the `transformed parameters` block as variable `gamma`:

The spatial effects parameters `phi` has an ICAR prior.  We implement this by defining a log probability density function to compute the ICAR model via the pairwise difference formula.  Because this is an improper prior, we must add a soft sum-to-zero constraint:

### Freni_Sterrantino recommendations for BYM2 model for disconnected graphs

Freni-Sterrantino et al show how to adjust the scaling factors when the areal map is not fully connected but has at least one connected multi-node component.  For a map with $K$ components:

- Each connected component of size > 1 is scaled independently with scaling factor ${\tau}_k$ and a sum-to-zero constraint is imposed on that component.

- Components of size 1 are drawn from a standard distribution; i.e., they are treated as having random i.i.d. spatial variance.

### Stan implementation of the BYM2 model for disconnected graphs

To extend the BYM2 model to these areal maps, we agument this model with a series of per-component masks into the node and edgelists and use Stan's multi-index operator and vectorized operations for efficient computation.

The spatial structure includes a set of arrays describing component-wise node, edgesets.
The `_cts` arrays record the size of the node and edgelists for each component, the `_idx` arrays provide the indices of the members of each component.

The BYM2 model parameters are the same; however the computation of the combined random effect `gamma` is computed component-wise:

The ICAR `_lpdf` function puts a sum-to-zero constraint on each component.
Singletons have normal spatial variance.

### Regression data:  the Scotland cancer dataset

The canonical dataset used to test and compare different parameterizations of ICAR models is a study on the incidence of lip cancer in Scotland in the 1970s and 1980s. The data, including the names and coordinates for the counties of Scotland are available from R package SpatialEpi, dataset scotland.

The cancer study data is a set of observations taken from 56 counties in Scotland.  It consists of 3 parallel arrays:

- `y`: observed outcome - number of cases of lip cancer
- `x`: single predictor - percent of population working in agriculture, forestry, or fisheries.
- `E`: population

This data is common to all of the `.data.json` files, also in file `scotland_lip.data.R`.  The Scotland lip cancer data is in many R packages and textbook examples.  The pre
dictor `x` is sometimes called `AFF`; it represents the proportion or percentage of the population employed in agriculture, farming, and fishing.  Sometimes the predictor `x` it is given as a percentage, i.e., scaled from 1 to 100; sometimes as a proportion, scaled from 0 to 1.  However in the BYM literature, in the regression, this factor is often the population percentage scaled by 1/10; in order to match reported estimates of the regression coefficients reported by BUGS and R-INLA, we scale the values of `x` to match.

### Areal data:  the 56 counties of Scotland

The common format for the spatial structure of an areal dataset is as a set of [shapefiles](https://en.wikipedia.org/wiki/Shapefile), a geospatial vector format.  The R package `spdep` provides methods which determines the adjacency relations between spatial polygons and returns a `nb` object which is list over the nodes in the graph, each element of which the list of the indices of neighboring nodes, or `0`, for singleton node.

For the BUGS implementation of the ICAR and BYM models, the spatial structure of the areal map is in the `nb` object format, i.e., a list of lists of node indices.  We have written a set of helper functions which take the nb objects for each graph into the set of data structures needed by the Stan models, these are in file bym2_helpers.R. The helper function `nb_to_edge_array` takes the nb object and returns the 2  ×  J edge array; the helper function `scaling_factor` uses the edge array to compute the geometric mean of the corresponding adjacency matrix, and the helper function index_components function creates the per-component node and edge index arrays, and computes the vector of scaling factors tau.

The `scotland` dataset contains the shapefiles for the counties in Scotland.  The fully connected graph corresponds to the data as distributed.  By editing the `nb` objects, we have created the alternative maps above.
The three versions of the Scotland spatial structure are in files `scotland_nbs.data.R`, `scotland_3_comp_nbs.data.R`, and `scotland_islands_nbs.data.R`.
The file `munge_scotland.R` munges the data, and it has been saved as JSON data files.

Three counties in Scotland are islands:  the Outer Hebrides (western.isles), Shetland, and Orkney.  In the canonical datasets, these islands are conntected to the mainland, so that the adjacency graph consists of a single, fully connected component.  However, different maps are possible:  a map with 4 components, the mainland and the 3 islands; or a map with 2 components where the 3 islands are connected to one another, distinct from the mainland, or various other combinations of the islands.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline


# figure size in inches optional
rcParams['figure.figsize'] = (25,20)
img_A = mpimg.imread('scot_connected.png')
img_B = mpimg.imread('scot_islands.png')
img_C = mpimg.imread('scot_3_comp.png')
# display images
fig, ax = plt.subplots(1,3)
ax[0].imshow(img_A);
ax[1].imshow(img_B);
ax[2].imshow(img_C);

The graph nodes which have different amounds of connectivity are:  1: "sky-lochalsh" , 3: "caithnes", 5: "ross-cromarty", 6: "shetland", 8: "orkney", 9: "moray", 11: "western.isles", 12: "sutherland".

### Fitting the BYM2_islands model to Scotland map as mainland component plus islands

To run CmdStanPy, we load the following libraries and packages:

In [ ]:
import json
import numpy as np
import pandas as pd

from cmdstanpy import cmdstan_path, CmdStanModel, install_cmdstan
# install_cmdstan()  # as needed - will install latest release (as needed)

The input data file `scotland_islands.data.json` was produced using R and our helper functions, working from file
file `scotland_islands_nbs.data.R`, which has the spatial structure as a graph with one connected component for the mainland and 3 island (singleton) components. This map has the same nodeset as in the `scotland` dataset, but the neighbors object (file `scotland_nbs.data.R`) has been edited to remove all edges between islands and the mainland or each other.

In [ ]:
with open('scotland_islands.data.json') as fd:
    islands_data = json.load(fd)

print('num nodes: {}, num edges: {}'.format(islands_data['I'], islands_data['J']))

print('num components: {}\nscaling factors: {}\nnodes per component: {}\nnode indices: {}'.format(
    islands_data['K'], islands_data['tau'], islands_data['K_node_cts'], islands_data['K_node_idxs']))

We use CmdStanPy to compile and fit the model using 

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel
bym2_islands_model = CmdStanModel(stan_file='bym2_islands.stan')

In [ ]:
print(bym2_islands_model.code())

We fit the model using the data dictionary `islands_data`, alternatively, one can specify file `scotland_islands.data.json`.  For reproducibility, we specify the seed supplied to the Stan pseudo-random number generator used by the sampler.

In [ ]:
bym2_islands_fit = bym2_islands_model.sample(data=islands_data, seed=12345)

The CmdStanMCMC method `summary` wraps the CmdStan [`stansummary`](https://mc-stan.org/docs/cmdstan-guide/stansummary.html) utility; it returns a pandas DataFrame object, one row per summary output row for the joint log probability density `lp__` and for the values of all variables in the Stan program.

In [ ]:
islands_summary = bym2_islands_fit.summary()
islands_summary

The R_hat and N_Eff (number of effective samples) in the summary report indicate that the model has fit the data.  To further check the fit, we run the `diagnose` method, which wraps CmdStan's [`diagnose`](https://mc-stan.org/docs/cmdstan-guide/diagnose.html) utility.

In [ ]:
bym2_islands_fit.diagnose()

The diagnose command detects _potential_ problems by examining the set of sampler diagnostic variables; these are the initial columns of the Stan CSV output file, which have column labels which end in `__`.  For example, to further check the treedepth, which is an integer output between 1 and sampler configuration argument `max_treedepth`, default 10, we can access this column from the output and do a quick summary of the per-draw treedepth counts:

In [ ]:
bym2_islands_draws = bym2_islands_fit.draws(concat_chains=True)
treedepth_idx = bym2_islands_fit.column_names.index('treedepth__')
treedepths = np.bincount(bym2_islands_draws[:, treedepth_idx].astype('int'))
for idx, val in enumerate(treedepths):
    if val > 0:
        print(idx, val)

To get the summary statistics for a Stan program variable across all chains, we can access individual rows of the pandas DataFrame object using [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html).  Here we examine the estimates for the quantities `phi` (spatial component) and `eta` (log relative risk) for the first 12 nodes in the graph:

In [ ]:
names = list(islands_summary.index)
phi_rows = [names.index(name) for name in names if name.startswith('phi[')]
eta_rows = [names.index(name) for name in names if name.startswith('eta[')]

In [ ]:
print('spatial effects:\n{}\n\nlog relative risk:\n{}\n'.format(
    islands_summary.iloc[phi_rows,:][0:11], 
    islands_summary.iloc[eta_rows,:][0:11]))

### Fitting the BYM2 model for a connected graph implemented in Stan.

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel, install_cmdstan
# install_cmdstan()  # as needed - will install latest release (as needed)

The dataset `scot_connected.data.json` contains the cancer dataset together with the spatial structure.
The spatial structure is comprised of:

- I: `int<lower = 0> I;  // number of nodes`
- J: `int<lower = 0> J;  // number of edges`
- edges: `int<lower = 1, upper = I> edges[2, J];  // node[1, j] adjacent to node[2, j]`
- tau: `real tau; // scaling factor`

In [ ]:
with open('scotland_connected.data.json') as fd:
    connected_data = json.load(fd)

bym2_model = CmdStanModel(stan_file='bym2.stan')
bym2_fit = bym2_model.sample(data=connected_data, seed=12345)

In [ ]:
connected_summary = bym2_fit.summary()
connected_summary

We compare the different estimates for model parameters `alpha`, `beta`, `rho`, and `sigma` for the connected and disconnected fits:

In [ ]:
print('connected\n{}\n\nislands\n{}'.format(connected_summary.iloc[1:5,:], islands_summary.iloc[1:5,:]))

For the connected and islands graphs, the nodes which have different connectivity are: (1,3,5,6,9,11,12).  To see how this affects the estimates, we look at the differences between the relative risks, using Arviz.

In [ ]:
import arviz as az
coords = {"region": np.arange(1,57)}
dims={"phi": ["region"], "eta": ["region"], "theta": ["region"], "gamma":["region"], "y_prime":["region"]}

connected_az = az.from_cmdstanpy(posterior=bym2_fit, coords=coords, dims=dims)
islands_az = az.from_cmdstanpy(posterior=bym2_islands_fit, coords=coords, dims=dims)

In [ ]:
az.rcParams.update({'plot.max_subplots': 56})

az.plot_density(
    [connected_az, islands_az], 
    data_labels=['connected', 'islands'], 
    var_names=["eta"],
    shade=0.1
)
plt.show()

We can also fit the connected graph as if it were a disconnected graph; the `BYM2_islands.stan` program allows graphs with a single component as input, all we need to do is supply the additional input data structures "K", "K_node_cts", etc.

In [ ]:
with open('scotland_connected_as_Kgraph.data.json') as fd:
    s2_data = json.load(fd)
bym2_islands_model = CmdStanModel(stan_file='bym2_islands.stan')
bym2_s2_fit = bym2_islands_model.sample(data=s2_data, seed=12345)

s2_summary = bym2_s2_fit.summary()

Both models returns the same estimates, although the islands model is slower, due to the overhead of the indexing operations.

In [ ]:
print('connected, fit with bym2_model\n{}\n\n'
      'connected, fit with bym2_islands model\n{}'.format(
          connected_summary.iloc[1:5,:], s2_summary.iloc[1:5,:]))

### Mainland plus 1 singleton, and combined Shetland + Orkney component

For the Scotland map with 3 components, in file `scotland_3_comps_nbs.data.R`, the islands of Shetland and Orkney have been combined.  This dataset exists to test additional possible kinds of maps, but provides no additional insights on the model or the domain data.

In [ ]:
bym2_c3_fit = bym2_islands_model.sample(data='scotland_3_comp.data.json', seed=12345)

In [ ]:
c3_summary = bym2_c3_fit.summary()
c3_summary